In [1]:
!pip -q install langchain huggingface_hub python-dotenv
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain

llm = HuggingFaceHub(
    repo_id="google/flan-t5-base",
    model_kwargs={"temperature": 0}
)

In [3]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT

from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate

from apis.food_list import API_DOCS

In [4]:
template = """
When the input content {input_text} is a noun related to food, please request the food list API and return its response content.
Otherwise, do not request any API, directly return the input content {input_text}\n\n
"""

api_prompt = PromptTemplate(
    input_variables=["input_text"], 
    template=template
)

api_chain = APIChain.from_llm_and_api_docs(llm, API_DOCS, prompt=api_prompt, verbose=True)

In [5]:
api_chain.run("sushi")



> Entering new APIChain chain...
api_url= http://localhost:3000/v1/foods
http://localhost:3000/v1/foods
["Salmon","Xiao Long Bao","Twice Cooked Pork","Lanzhou noodles"]

> Finished chain.


"['Sushi', 'Xiao Long Bao', "

In [6]:
result_prompt = PromptTemplate(
    input_variables=["api_response"],
    template="The food you need is :\n\n{api_response}"
)

result_chain = LLMChain(llm=llm, prompt=result_prompt, verbose=True)

In [7]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[api_chain, result_chain], verbose=True)

In [8]:
overall_chain.run("sushi")



> Entering new SimpleSequentialChain chain...


> Entering new APIChain chain...
api_url= http://localhost:3000/v1/foods
http://localhost:3000/v1/foods
["Salmon","Xiao Long Bao","Twice Cooked Pork","Lanzhou noodles"]

> Finished chain.
['Sushi', 'Xiao Long Bao', 


> Entering new LLMChain chain...
Prompt after formatting:
The food you need is :

['Sushi', 'Xiao Long Bao', 

> Finished chain.
Xiao Long Bao

> Finished chain.


'Xiao Long Bao'

In [11]:
overall_chain.run("Hamburger")



> Entering new SimpleSequentialChain chain...


> Entering new APIChain chain...
api_url= Hamburger
Hamburger


MissingSchema: Invalid URL 'Hamburger': No scheme supplied. Perhaps you meant http://Hamburger?